In [1]:
from diffusers import StableDiffusionPipeline, LCMScheduler
import torch

model_id = "runwayml/stable-diffusion-v1-5"
lora_id = "latent-consistency/lcm-lora-sdv1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.load_lora_weights(lora_id)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.35.2",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [6]:
import os

image = pipe(
    "An abstract diagram of network connections on a dark background with thin green lines connecting data nodes. The connection points of the data lines gently pulse, creating the feeling of a living, constantly learning algorithm. 4k, high resolution, detailed, intricate, glowing nodes, dynamic flow, cyberpunk aesthetic, glowing trails, soft ambient lighting, cinematic, sharp focus, no text, no people.",
    num_inference_steps=30,
    guidance_scale=4
).images[0]
image.save("background.png")

os.system("./realesrgan-ncnn-vulkan -i background.png -o background.png -n realesrnet-x4plus")

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']


  0%|          | 0/30 [00:00<?, ?it/s]

[0 NVIDIA GeForce RTX 3060]  queueC=2[8]  queueG=0[16]  queueT=1[2]
[0 NVIDIA GeForce RTX 3060]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[0 NVIDIA GeForce RTX 3060]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 NVIDIA GeForce RTX 3060]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1
fopen /home/dusty/hackotons/human-segmentation-with-personal-backgrounds/Background_gen/models/realesrnet-x4plus.param failed
fopen /home/dusty/hackotons/human-segmentation-with-personal-backgrounds/Background_gen/models/realesrnet-x4plus.bin failed
find_blob_index_by_name data failed
Try
find_blob_index_by_name output failed
Try
0.00%
find_blob_index_by_name data failed
Try
find_blob_index_by_name output failed
Try
11.11%
find_blob_index_by_name data failed
Try
find_blob_index_by_name output failed
Try
22.22%
find_blob_index_by_name data failed
Try
find_blob_index_by_name output failed
Try
33.33%
find_blob_index_by_name data failed
Try
find_blob_index_by_name output failed
Try
44.44%
find_blob_index_by_name

0

In [45]:
import json
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO

# --- Конфигурация ---
FONT_PATH_SANSATION = "/home/dusty/hackotons/human-segmentation-with-personal-backgrounds/Background_gen/Sansation-Bold.ttf"
BACKGROUND_IMAGE_PATH = "/home/dusty/Downloads/7ixg1BJM_Кейс Цифровой дресс-код_2025_10_23_0905_50.701080_+0000/Подложки DION/1920х1080.png"

def load_font(size):
    try:
        return ImageFont.truetype(FONT_PATH_SANSATION, int(size))
    except OSError:
        print(f"Шрифт {FONT_PATH_SANSATION} не найден, используем дефолтный.")
        return ImageFont.load_default()

def load_image_from_url(url):
    try:
        response = requests.get(url.strip())
        response.raise_for_status()
        return Image.open(BytesIO(response.content))
    except Exception as e:
        print(f"Ошибка загрузки изображения {url}: {e}")
        return None

def generate_background(employee_data):
    font_large = load_font(32)
    font_medium = load_font(24)
    font_small = load_font(16)
    font_tiny = load_font(12)

    img = Image.open(BACKGROUND_IMAGE_PATH)
    img = img.resize((1920, 1080), Image.LANCZOS)

    draw = ImageDraw.Draw(img, "RGBA")

    level = employee_data.get("privacy_level", "low")
    display_data = {}
    if level in ["low", "medium", "high"]:
        display_data.update({
            "full_name": employee_data.get("full_name"),
            "position": employee_data.get("position")
        })
    if level in ["medium", "high"]:
        display_data.update({
            "company": employee_data.get("company"),
            "department": employee_data.get("department"),
            "office_location": employee_data.get("office_location")
        })
    if level == "high":
        contact = employee_data.get("contact", {})
        branding = employee_data.get("branding", {})
        display_data.update({
            "email": contact.get("email"),
            "telegram": contact.get("telegram"),
            "logo_url": branding.get("logo_url"),
            "slogan": branding.get("slogan")
        })

    branding = employee_data.get("branding", {})
    primary_hex = branding.get("corporate_colors", {}).get("primary", "#0052CC")
    secondary_hex = branding.get("corporate_colors", {}).get("secondary", "#00B8D9")
    primary_color = tuple(int(primary_hex[i:i+2], 16) for i in (1, 3, 5))
    secondary_color = tuple(int(secondary_hex[i:i+2], 16) for i in (1, 3, 5))
    primary_color = (0, 82, 204)
    secondary_color = (0, 184, 217)

    # --- Рендеринг текста ---
    block1_x, block1_y = 64, 56
    if display_data.get("full_name"):
        draw.rounded_rectangle([38, 36, 575 + 38, 100 + 36], fill="white", radius=15)
        draw.text((block1_x, block1_y), display_data["full_name"], fill=primary_color, font=font_large)
    if display_data.get("position"):
        draw.text((block1_x, block1_y + 40), display_data["position"], fill=secondary_color, font=font_small)

    block2_x, block2_y = 64, 163
    if display_data.get("company"):
        draw.rounded_rectangle([38, 144, 370 + 38, 425 + 144], fill="white", radius=15)
        draw.text((block2_x, block2_y), display_data["company"], fill=primary_color, font=font_medium)
    if display_data.get("department"):
        draw.text((block2_x, block2_y + 35), display_data["department"], fill=secondary_color, font=font_small)

    email_x, email_y = 1551, 48
    if level == "high" and display_data.get("email"):
        draw.rounded_rectangle([1500, 41, 384 + 1500, 50 + 41], fill="white", radius=15)
        draw.rounded_rectangle([1503, 44, 44 + 1503, 44 + 44], fill="#00B8D9", radius=15)
        draw.text((email_x, email_y), display_data["email"], fill=primary_color, font=font_large)
        draw.text((1508, 46), "@", fill=primary_color, font=font_large)

    location_x, location_y = 90, 592
    if display_data.get("office_location"):
        draw.rounded_rectangle([38, 577, 384 + 38, 50 + 577], fill="white", radius=15)
        draw.rounded_rectangle([40, 580, 44 + 40, 44 + 580], fill="#00B8D9", radius=15)
        draw.text((location_x, location_y), display_data["office_location"], fill=primary_color, font=font_small)
        draw.image((location_x, location_y), display_data["office_location"], fill=primary_color, font=font_small)

    # --- Блок 3: Логотип ---
    block3_x, block3_y, block3_width, block3_height = 1531, 117, 332, 332
    draw.rectangle([block3_x, block3_y, block3_x + block3_width, block3_y + block3_height], fill=primary_color)
    if level == "high" and display_data.get("logo_url"):
        logo_img = load_image_from_url(display_data["logo_url"])
        if logo_img:
            logo_img.thumbnail((block3_width, block3_height))
            logo_x = block3_x + (block3_width - logo_img.width) // 2
            logo_y = block3_y + (block3_height - logo_img.height) // 2
            img.paste(logo_img, (logo_x, logo_y), logo_img.convert('RGBA') if logo_img.mode in ('RGBA', 'LA') else None)

    # --- Блок 4: Контакты ---
    telegram_x, telegram_y = 1535, 480
    if level == "high" and display_data.get("telegram"):
        draw.rounded_rectangle([1508, 100, 375 + 1508, 425 + 100], fill="white", radius=15)
        draw.text((telegram_x, telegram_y), display_data["telegram"], fill=primary_color, font=font_tiny)

    # --- Блок 5: Слоган ---
    slogan_x, slogan_y = 1523, 543
    if level == "high" and display_data.get("slogan"):
        draw.rounded_rectangle([1508, 530, 375 + 1508, 50 + 530], fill="white", radius=15)
        draw.text((slogan_x, slogan_y), f"«{display_data['slogan']}»", fill=primary_color, font=font_medium)

    return img.convert("RGB")

# --- Пример использования ---
if __name__ == "__main__":
    sample_json_str = """
    {
      "employee": {
        "full_name": "Иванов Сергей Петрович",
        "position": "Ведущий инженер по компьютерному зрению",
        "company": "ООО «Рога и Копыта»",
        "department": "Департамент компьютерного зрения",
        "office_location": "Новосибирск, технопарк «Идея»",
        "contact": {
          "email": "sergey.ivanov@t1dp.ru",
          "telegram": "@sergey_vision"
        },
        "branding": {
          "logo_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Google_2015_logo.svg/250px-Google_2015_logo.svg.png",
          "corporate_colors": {
            "primary": "#4285F4",
            "secondary": "#34A853"
          },
          "slogan": "Инновации в каждый кадр"
        },
        "privacy_level": "high"
      }
    }
    """

    try:
        data = json.loads(sample_json_str)
        employee_info = data["employee"]
        background_image = generate_background(employee_info)
        output_filename = f"personalized_background_{employee_info['privacy_level']}.jpg"
        background_image.save(output_filename, "JPEG", quality=100)
        print(f"Фон сохранен как {output_filename}")
    except json.JSONDecodeError as e:
        print(f"Ошибка парсинга JSON: {e}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

Ошибка загрузки изображения https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Google_2015_logo.svg/250px-Google_2015_logo.svg.png: 403 Client Error: Forbidden for url: https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Google_2015_logo.svg/250px-Google_2015_logo.svg.png
Фон сохранен как personalized_background_high.jpg
